# TP Technique

Dans ce notebook, nous allons explorer plusieurs techniques avancées pour l'entraînement de modèles :
- PyTorch Lightning pour structurer notre code d'entraînement
- Weights & Biases (wandb) pour le suivi des expériences
- Data augmentation pour améliorer la robustesse
- Transfer learning pour tirer parti des modèles pré-entraînés


# Données

On va télécharger les données MNIST et les charger dans un DataLoader.


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset

## Data Augmentation

In [ ]:
transform_augmented = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.RandomRotation(10),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.RandomErasing(p=0.1),
])

train_dataset_augmented = datasets.MNIST(
    root='./data', 
    train=True,
    download=True,
    transform=transform_augmented
)

## Data Loading

In [ ]:
import os
class MNISTData(Dataset):
    def __init__(self, dataset_dir = './data', train=True):
        self.dataset_dir = dataset_dir
        self.len = len(os.listdir(self.dataset_dir))
        self.train = train
    def __len__(self):
        return self.len
        
    
    def __getitem__(self, index):
        image_path = os.path.join(self.dataset_dir, 'train' if self.train else 'test', f"{index}.png")
        image = Image.open(image_path)
        return image

train_loader = DataLoader(MNISTData(dataset_dir='./data/train'), batch_size=64, shuffle=True)
test_loader = DataLoader(MNISTData(dataset_dir='./data/test'), batch_size=64, shuffle=True)

# Pytorch Lightning

In [ ]:
from torch import optim, nn, utils
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import torch.nn.functional as F
import wandb

wandb.login()

## Définition du modèle

In [ ]:
class MNISTModel(pl.LightningModule):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('train_loss', loss)
        self.log('train_acc', (y_hat.argmax(dim=1) == y).float().mean())
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('val_loss', loss)
        self.log('val_acc', (y_hat.argmax(dim=1) == y).float().mean())
        return loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('test_loss', loss)
        self.log('test_acc', (y_hat.argmax(dim=1) == y).float().mean())
        return loss
    
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)
    

## Entrainement

In [ ]:
trainer = pl.Trainer(
    max_epochs=10, 
    gpus=1, 
    logger=WandbLogger(project='tp_technique'), 
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=3),
        ModelCheckpoint(monitor='val_loss', mode='min', save_top_k=1, filename='best_model')
    ]
)

model = MNISTModel()
trainer.fit(model, train_loader, test_loader)

## Evaluation
trainer.test(model, test_loader)